In [3]:
import tensorflow as tf


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [1]:
import tensorflow as tf

USE_GPU = True

if USE_GPU and tf.config.list_physical_devices('GPU'):
    device = 'GPU'
    print("Using GPU")
else:
    device = 'CPU'
    print("Using CPU")

Using CPU


In [3]:
import tensorflow_datasets as tfds

# Load the ImageNet dataset

data_dir = '/home/asko/tensorflow_datasets/downloads/manual'

# Load the ImageNet Resized dataset (e.g., 32x32)
dataset, info = tfds.load('imagenet_resized/64x64', data_dir=data_dir, with_info=True, as_supervised=True)

# Split the dataset into training and validation sets
train_dataset = dataset['train']
val_dataset = dataset['validation']

# Print the dataset info
print(info)

/home/asko/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tfds.core.DatasetInfo(
    name='imagenet_resized',
    full_name='imagenet_resized/64x64/0.1.0',
    description="""
    This dataset consists of the ImageNet dataset resized to fixed size. The images
    here are the ones provided by Chrabaszcz et. al. using the box resize method.
    
    For [downsampled ImageNet](http://image-net.org/download.php) for unsupervised
    learning see `downsampled_imagenet`.
    
    those from the other ImageNet datasets provided by Tensorflow datasets. See the
    original
    [label list](https://github.com/PatrykChrabaszcz/Imagenet32_Scripts/blob/master/map_clsloc.txt),
    and the
    [labels used by this dataset](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/image_classification/imagenet_resized_labels.txt).
    Additionally, the original authors 1 index there labels which we convert to 0
    indexed by subtracting one.
    """,
    config_description="""
    Images resized to 64x64
    """,
    homepage='https://patrykc

2024-11-15 18:48:19.540097: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-15 18:48:19.541578: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-15 18:48:19.541893: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-15 18:48:19.542032: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), bu

In [2]:
# from keras.applications import VGG16
from VGG16.vgg import VGG
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

alpha = 0.03
temperature = 0.02

# Load the VGG16 model
model = VGG(alpha=alpha, temperature=temperature)


In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg16 import preprocess_input

data_dir = '/home/asko/tensorflow_datasets/downloads/manual'
dataset, info = tfds.load('imagenet_resized/64x64', data_dir=data_dir, with_info=True, as_supervised=True)

train_dataset = dataset['train']
val_dataset = dataset['validation']


def preprocess(image, label):
    image = tf.image.resize(image, [64, 64])
    image = preprocess_input(image) 
    return image, label

batch_size = 32
train_dataset = (train_dataset
                 .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
                 .shuffle(1000)  
                 .batch(batch_size)
                 .prefetch(tf.data.AUTOTUNE))

val_dataset = (val_dataset
               .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
               .batch(batch_size)
               .prefetch(tf.data.AUTOTUNE))

In [3]:
import tarfile

# Path to the downloaded tgz file
tgz_path = "/Users/apple/imagenette2-320.tgz"
extract_path = "./imagenette"  # Target folder for extraction

# Extract the file
with tarfile.open(tgz_path, "r:gz") as tar:
    tar.extractall(path=extract_path)
print("Extraction completed.")

Extraction completed.


In [4]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler

data_dir = '/Users/apple/Documents/MSCS/Fall 2024/CS682/Project/cs682-knowledge-distillation/imagenette2-320.tgz'

transform = transforms.Compose([
    transforms.Resize((227,227)),  # Resize images to a size suitable for VGG16
    # transforms.CenterCrop(224),  # Crop to the expected input size for VGG16
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize as per VGG16
])
extract_path = "./imagenette"
dataset = datasets.ImageFolder(
    root=extract_path,  # Imagenette URL
    transform=transform
)
# subset_indices = list(range(16000))  
# train_subset = Subset(train_dataset, subset_indices)

batch_size = 50
sampler = SubsetRandomSampler(indices)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# for images, labels in train_loader:
    # print(labels.shape)


In [5]:
for images, labels in train_loader:
    print(f"Batch of images shape: {images.shape}")  # Shape: [batch_size, 3, 64, 64]
    print(images)
    print(f"Batch of labels shape: {labels.shape}")  # Shape: [batch_size]
    print(labels)
    break

Batch of images shape: torch.Size([50, 3, 227, 227])
tensor([[[[-1.5528, -1.5699, -1.5699,  ..., -1.0733, -1.1075, -1.1418],
          [-1.5699, -1.5699, -1.5699,  ..., -1.0733, -1.0904, -1.1418],
          [-1.5699, -1.5699, -1.5699,  ..., -1.0733, -1.0904, -1.1247],
          ...,
          [-1.5870, -1.5870, -1.5870,  ..., -1.6727, -1.5357, -1.6898],
          [-1.5870, -1.5870, -1.5870,  ..., -1.6727, -1.5357, -1.6898],
          [-1.5870, -1.5870, -1.5870,  ..., -1.6727, -1.5357, -1.6898]],

         [[-1.1954, -1.1779, -1.1779,  ..., -0.2325, -0.2675, -0.3025],
          [-1.1779, -1.1779, -1.1779,  ..., -0.2325, -0.2500, -0.3025],
          [-1.1779, -1.1779, -1.1779,  ..., -0.2500, -0.2675, -0.3025],
          ...,
          [-1.4930, -1.4930, -1.4930,  ..., -1.5805, -1.4405, -1.5980],
          [-1.4930, -1.4930, -1.4930,  ..., -1.5805, -1.4405, -1.5980],
          [-1.4930, -1.4930, -1.4930,  ..., -1.5805, -1.4405, -1.5980]],

         [[-0.4624, -0.4624, -0.4624,  ...,  1.03

In [6]:
import tensorflow as tf
from VGG16.vgg import VGG

alpha = 0.03
temperature = 0.02

# Load the VGG16 model
model = VGG(alpha=alpha, temperature=temperature)

device = 'cuda' if len(tf.config.list_physical_devices('GPU'))!=0 else 'cpu'
model.to(device)
print(device)

cpu


In [7]:
def check_accuracy_part34(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    print(device)
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            print(y)
            print(preds)
            # print(scores)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        # print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return 100 * acc


In [7]:
def check_accuracy_part35(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # Set model to evaluation mode
    print(device)
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=torch.float32)  # Move inputs to device
            y = y.to(device=device, dtype=torch.long)  # Move labels to device, long for classification
            
            scores = model(x)  # Get model predictions
            _, preds = scores.max(1)  # Get predicted class indices
            
            num_correct += preds.eq(y).sum().item()  # Compare predictions with ground truth
            num_samples += preds.size(0)  # Count samples

        acc = float(num_correct) / num_samples
        return 100 * acc


In [ ]:
import torch.nn.functional as F
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(train_loader):
            model.train()  # put model to training mode
            x = x.to(device=device)  # move to device, e.g. GPU
            y = y.to(device=device)
            
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            if t % 100 == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                # check_accuracy_part34(loader_val, model)
                print()

In [8]:
def one_hot_encode(labels, num_classes=1000):
    # Initialize a tensor of zeros with the shape [batch_size, num_classes]
    one_hot_labels = torch.zeros(labels.size(0), num_classes).cuda()
    one_hot_labels.scatter_(1, labels.view(-1, 1), 1)  # Set 1 at the correct index
    return one_hot_labels

In [20]:
check_accuracy_part34(train_loader, model)

cuda
Got 1300 / 16000 correct (8.12)
